<a href="https://colab.research.google.com/github/mahdieslaminet/ML-Final-Project/blob/main/FutsalTalentApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =============================
# FUTSAL TALENT SCOUT APP - Colab Version Unified Input
# =============================

!pip install streamlit pyngrok opencv-python-headless matplotlib tensorflow scikit-learn --quiet

app_code = """
import streamlit as st
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Concatenate
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler

st.set_page_config(page_title='Futsal Talent Scout AI', layout='wide')
st.title('⚽ Futsal Talent Scout AI')

# --- توضیحات ---
st.markdown(\"\"\"
### 📝 How to Use This App

**Step 1 – Enter Text Information (Left Panel 👈)**
All **player physical and skill information** must be entered in the **sidebar on the left**:
- Height & Weight
- Starting Age
- Speed, Agility, Endurance
- Passing, Dribbling, Shooting

⬅️ **Look to the left side of the screen to enter these values**

---

**Step 2 – Upload Player Image (Optional)**
You can upload a player image below.
- If no image is uploaded, the AI will analyze the player **using text data only**
- If an image is uploaded, **both text + image** are used for analysis

---

**Step 3 – Analyze**
Click **Analyze Player** to:
- Check futsal suitability
- Get suggested playing position
- View skill radar chart
\"\"\")

# --- فرم ورودی ---
st.sidebar.header("Player Information (Text)")
height = st.sidebar.number_input("Height (m)", min_value=1.4, max_value=2.2, value=1.75)
weight = st.sidebar.number_input("Weight (kg)", min_value=40, max_value=120, value=70)
start_age = st.sidebar.number_input("Professional Start Age", min_value=10, max_value=40, value=18)
speed = st.sidebar.slider("Speed (1-10)", 1, 10, 7)
agility = st.sidebar.slider("Agility (1-10)", 1, 10, 7)
endurance = st.sidebar.slider("Endurance (1-10)", 1, 10, 7)
passing = st.sidebar.slider("Passing Skill (1-10)", 1, 10, 7)
dribbling = st.sidebar.slider("Dribbling Skill (1-10)", 1, 10, 7)
shooting = st.sidebar.slider("Shooting Skill (1-10)", 1, 10, 7)

st.header("Upload Player Image (Optional)")
uploaded_file = st.file_uploader("Choose Image", type=['jpg','jpeg','png'])
img_ready = None
if uploaded_file:
    file_bytes = np.asarray(bytearray(uploaded_file.read()), dtype=np.uint8)
    img = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)
    img_resized = cv2.resize(img, (224,224))
    img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB)
    img_ready = np.expand_dims(img_rgb/255.0, axis=0)
    st.image(img_rgb, caption="Uploaded Player Image", use_column_width=True)

# --- آنالیز ---
if st.button("Analyze Player"):

    st.subheader("Analyzing Player...")

    # تصویر آماده است؟
    if img_ready is not None:
        base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))
        x = base_model.output
        x = Flatten()(x)
        image_model = Model(inputs=base_model.input, outputs=x)
        image_features = image_model.predict(img_ready)
    else:
        image_features = np.zeros((1, 7*7*1280))  # placeholder بدون تصویر

    # داده متنی
    user_vector = np.array([[height, weight, start_age, speed, agility, endurance, passing, dribbling, shooting]])
    scaler = MinMaxScaler()
    user_text_scaled = scaler.fit_transform(user_vector)

    # مدل ترکیبی
    input_text = Input(shape=(user_text_scaled.shape[1],))
    input_image = Input(shape=(image_features.shape[1],))
    combined = Concatenate()([input_text, input_image])
    x = Dense(128, activation='relu')(combined)
    x = Dense(64, activation='relu')(x)
    output = Dense(6, activation='softmax')(x)

    final_model = Model(inputs=[input_text, input_image], outputs=output)
    final_model.compile(optimizer=Adam(0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # آموزش سریع نمونه برای نمایش
    X_text_scaled = np.repeat(user_text_scaled, 6, axis=0)
    X_image_features = np.repeat(image_features, 6, axis=0)
    y_dummy = np.arange(6)
    final_model.fit([X_text_scaled, X_image_features], y_dummy, epochs=3, verbose=0)

    # پیش‌بینی پست
    pred = final_model.predict([user_text_scaled, image_features])
    pred_class = np.argmax(pred)
    positions = ["Goalkeeper","Defender","Midfielder","Winger","Forward","Pivot"]

    # بررسی تناسب فوتسال
    MIN_HEIGHT, MAX_HEIGHT = 1.60, 2.00
    skills_avg = np.mean([speed, agility, endurance, passing, dribbling, shooting])

    if MIN_HEIGHT <= height <= MAX_HEIGHT and 18 <= start_age <= 30 and skills_avg >= 6:
        st.success("Player is suitable for futsal ⚽")
        st.info(f"Suggested Position: {positions[pred_class]}")
    else:
        st.error("Player is NOT suitable for futsal ❌")

    # نمودار راداری مهارت‌ها
    skills = [speed, agility, endurance, passing, dribbling, shooting]
    skill_labels = ["Speed","Agility","Endurance","Passing","Dribbling","Shooting"]
    angles = np.linspace(0, 2*np.pi, len(skills), endpoint=False).tolist()
    skills += skills[:1]
    angles += angles[:1]

    fig, ax = plt.subplots(figsize=(5,5), subplot_kw=dict(polar=True))
    ax.plot(angles, skills, 'o-', linewidth=2, label="Player Skills")
    ax.fill(angles, skills, alpha=0.25)
    ax.set_thetagrids(np.degrees(angles[:-1]), skill_labels)
    ax.set_ylim(0,10)
    st.pyplot(fig)
"""

# ذخیره کد اپ
with open("/content/futsal_colab_app.py", "w") as f:
    f.write(app_code)

# =============================
# اجرای Streamlit با ngrok
# =============================
from pyngrok import ngrok
import os

ngrok.kill()  # پایان دادن تونل های قبلی
!ngrok authtoken 37a1eCfi8Rt8zXtYBLkxmLaroGK_37eQpe3cfFhFxetLHf1EF  # جایگزین توکن Verify شده خودت

port = 8501
public_url = ngrok.connect(port)
print(f"🌍 Open your app on: {public_url}")

os.system(f"streamlit run /content/futsal_colab_app.py --server.port {port} --server.headless true")

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
🌍 Open your app on: NgrokTunnel: "https://unpigmented-uncounselled-crew.ngrok-free.dev" -> "http://localhost:8501"
